In [9]:
from torchvision import models
import torch 
import torch.onnx

In [12]:
def create_network():
  model = models.resnet18()
  input_dim = model.fc.in_features
  model.fc = torch.nn.Linear(input_dim,2)
  return model

In [13]:
model = create_network()

In [19]:
parameter = torch.load("/content/taco_burrito.prm" ,map_location="cpu")
model.load_state_dict(parameter)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [20]:
data = torch.empty(1,3,224,224,dtype=torch.float32)
torch.onnx.export(model,               # 실행될 모델
                  data,                         # 모델 입력값 (튜플 또는 여러 입력값들도 가능)
                  "taco_burrito.onnx",   # 모델 저장 경로 (파일 또는 파일과 유사한 객체 모두 가능)
                  export_params=True,        # 모델 파일 안에 학습된 모델 가중치를 저장할지의 여부
                  ##opset_version=10,          # 모델을 변환할 때 사용할 ONNX 버전
                  do_constant_folding=True,  # 최적화시 상수폴딩을 사용할지의 여부
                  input_names = ['input'],   # 모델의 입력값을 가리키는 이름
                  output_names = ['output'], # 모델의 출력값을 가리키는 이름
                  )

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [23]:
##test 이미지 구축
from PIL import Image
from torchvision import transforms

transform =  transforms.Compose([
     transforms.CenterCrop(224),
     transforms.ToTensor()
     ])
img = Image.open("/content/360.jpg")
img_tensor = transform(img).unsqueeze(0)
img_ndarray = img_tensor.numpy()

In [33]:
##pytorch 모델의 출력값
torch_out = model.forward(img_tensor)

In [41]:
##ONNX 모델의 출력값
import onnx
import onnxruntime
import numpy as np 
ort_session = onnxruntime.InferenceSession("taco_burrito.onnx")

# ONNX 런타임에서 계산된 결과값
ort_inputs = {ort_session.get_inputs()[0].name:img_ndarray}
ort_outs = ort_session.run(None, ort_inputs)

# ONNX 런타임과 PyTorch에서 연산된 결과값 비교
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)
print("Exported model has been tested with ONNXRuntime, and the result looks good!")


Exported model has been tested with ONNXRuntime, and the result looks good!


In [43]:
to_numpy(torch_out)

array([[ 1.5829802, -1.3070925]], dtype=float32)

In [44]:
ort_outs[0]

array([[ 1.5829809, -1.3070942]], dtype=float32)